# pyspark4
Работаем со spark sql

In [ ]:
import os
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

os.environ["HADOOP_CONF_DIR"]="/etc/hadoop/conf"
os.environ["HIVE_HOME"]="/usr/lib/hive"
os.environ["METASTORE_PORT"]="9083"
# os.environ["SPARK_HOME"]="/usr/hdp/current/spark2-client"
# os.environ["JAVA_HOME"]="/usr/java/jdk1.8.0_191/jre"

spark = pyspark.sql.SparkSession.builder\
    .master("yarn")\
    .appName("v_alehin_pyspark4")\
    .config("spark.executor.instances", "1")\
    .config("spark.executor.memory", "1G")\
    .config("spark.executor.cores", "2")\
    .config("spark.dynamicAllocation.enabled", "false")\
    .config("spark.dynamicAllocation.executorIdleTimeout", "300s")\
    .config("spark.dynamicAllocation.maxExecutors", "1000")\
    .config("spark.driver.memory", "1G")\
    .config("spark.driver.maxResultSize", "1G")\
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")\
    .config("spark.kryoserializer.buffer.max", "1024m")\
    .enableHiveSupport()\
    .getOrCreate()

spark_context = spark.sparkContext

In [ ]:
print("Yarn       http://91.219.226.252:8088/cluster/scheduler")
print("Spark      http://91.219.226.252:8088/proxy/{}/stages/".format(spark_context.applicationId))
print("App info   http://91.219.226.252:8088/cluster/app/{}/".format(spark_context.applicationId))

In [ ]:
sourceDF = (
    spark.read
        .option("header", "false")
        .option("sep", "\t")
        .csv("/user/v.alehin/data2/ok/coreDemography")
)

In [ ]:
sourceDF.printSchema()

In [ ]:
sourceDF.show()

In [ ]:
sourceDF = spark.read.table("v_alehin.core_demography")

In [ ]:
sourceDF.printSchema()

In [ ]:
spark.sql("""
SELECT 
    month(from_unixtime(birth_date * 24 * 3600)) AS month,
    count(1) as cnt_per_month
FROM v_alehin.core_demography
GROUP BY month(from_unixtime(birth_date * 24 * 3600))
ORDER BY month
""").show(20, False)

In [ ]:
# После работы обязательно отключаем спарк и отдаем ресурсы!
spark.stop()